In [164]:
import pandas as pd
import numpy as np
import os
file='./data/voynich.txt'

In [165]:
#df = pd.read_csv(file,header=None, sep='\t',engine='python',names=['image','description'])
df = pd.read_csv(file,header=None,sep='>',names=['heading','text'])
# df[['image','did']] = df.image.str.split("#",expand=True,)


print(len(df))
df.head(3)

5214


,heading,text
0,<f1r.P1.1;H,fachys.ykal.ar.ataiin.shol.shory.cthres...
1,<f1r.P1.2;H,sory.ckhar.or.y.kair.chtaiin.shar.are.c...
2,<f1r.P1.3;H,syaiir.sheky.or.ykaiin.shod.cthoary.cth...


In [166]:
df['heading'] = df.heading.str.replace('<','')
df['text'] = df.text.str.strip()
df.head(5)

,heading,text
0,f1r.P1.1;H,fachys.ykal.ar.ataiin.shol.shory.cthres.y.kor....
1,f1r.P1.2;H,sory.ckhar.or.y.kair.chtaiin.shar.are.cthar.ct...
2,f1r.P1.3;H,syaiir.sheky.or.ykaiin.shod.cthoary.cthes.dara...
3,f1r.P1.4;H,ooiin.oteey.oteos.roloty.cth*ar.daiin.otaiin.o...
4,f1r.P1.5;H,dair.y.chear.cthaiin.cphar.cfhaiin


In [167]:
df[['page','paragraph','line']] = df.heading.str.split(".",expand=True)

In [168]:
df['page']=df['page'].str.replace('f','')
df['text']=df['text'].str.replace('.',' ')
df['text']=df['text'].str.replace('=',' ')

C:\Users\bakig\AppData\Local\Temp\ipykernel_34104\2797887583.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['text']=df['text'].str.replace('.',' ')


In [169]:
df.head(3)

,heading,text,page,paragraph,line
0,f1r.P1.1;H,fachys ykal ar ataiin shol shory cthres y kor ...,1r,P1,1;H
1,f1r.P1.2;H,sory ckhar or y kair chtaiin shar are cthar ct...,1r,P1,2;H
2,f1r.P1.3;H,syaiir sheky or ykaiin shod cthoary cthes dara...,1r,P1,3;H


In [ ]:
df.to_csv('data.csv')

In [ ]:
data=df['text']

Creating our own tokenizer

In [ ]:
def get_training_corpus():
    for i in range(0, len(df)):
        yield df.iloc[i : ]["text"]

In [ ]:
import re
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, trainers, processors, decoders
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))



In [ ]:
tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace((r"[\p{Other}&&[^\n\t\r]]"), ""),
        normalizers.Replace((r"[\s]"), " "),
        normalizers.Lowercase(),
        normalizers.NFD(), normalizers.StripAccents()]
)

In [ ]:
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [ ]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [ ]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [ ]:
tokenizer.save('token_voynich.json')

In [170]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [182]:
df.to_csv('df.csv')

In [ ]:

txt_files_dir='./data/tokenfiles/'

In [172]:
#Group by paragraph
#datadf=df.groupby(['page','paragraph'])['text'].apply(lambda x: ''.join(x)).reset_index()

#group by page
datadf=df.groupby(['page'])['text'].apply(lambda x: ''.join(x)).reset_index()

In [173]:
datadf['size']=datadf['text'].str.len()

In [158]:
datadf.sort_values(by=['page'],inplace=True)


In [174]:
data=datadf['text']
datadf

,page,text,size
0,100r,chosarosholsochorcfhyotearchofarysar char-daii...,717
1,100v,tolchdcholsopchorrolsysol eesosykchochdyykchdy...,572
2,101r1,pcheol cheol ol shey qockhol shor yteol sheock...,1272
3,101v2,sairalyotaldyotalytadokororarotorarotalasoraly...,1277
4,102r1,dordodorolalydardshotodeeodorpolaiin shocthy q...,707
...,...,...,...
220,96v,psheoas sheeor qoepsheody odar ocpheo-opar ysa...,382
221,99r,okoramogokarydararokysalooro-ainokorsalolskeea...,1127
222,99v,otaramyotoldyotor-chyoldydar-aryotalyolsyarolo...,1073
223,9r,tydlo choly cthor orchey s shy odaiin sary-sho...,484


In [ ]:
# Store values in a dataframe column (Series object) to files, one file per record
def column_to_files(column, prefix, txt_files_dir):
    # The prefix is a unique ID to avoid to overwrite a text file
    i=prefix
    #For every value in the df, with just one column
    for row in column.to_list():
      # Create the filename using the prefix ID
      file_name = os.path.join(txt_files_dir, str(i)+'.txt')
      try:
        # Create the file and write the column text to it
        f = open(file_name, 'wb')
        f.write(row.encode('utf-8'))
        f.close()
      except Exception as e:  #catch exceptions(for eg. empty rows)
        print(row, e) 
      i+=1
    # Return the last ID
    return i
# Get the training data

# Removing the end of line character \n
data = data.replace("\n"," ")
# Set the ID to 0
prefix=0
# Create a file for every description value
prefix = column_to_files(data, prefix, txt_files_dir)
# Print the last ID
print(prefix)
# Get the test data


In [ ]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from pathlib import Path


paths = [str(x) for x in Path(".").glob("./data/tokenfiles/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(lowercase=True)

# Customize training
tokenizer.train(files=paths, vocab_size=8192, min_frequency=2,
                show_progress=True,
                special_tokens=[
                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>",
])
#Save the Tokenizer to disk
tokenizer.save_model('./data/')

    


In [161]:
data

0      chos!aroshol sochorcfhy otear chofary sar char...
1      tolchd chols opchor rolsy sol eesos ykchochdy ...
2      pcheol cheol ol shey qockhol shor yteol sheock...
3      sairaly otaldy otal yta dokor orar otorar otal...
4      dordod orolaly dardsh otodeeodor polaiin shoct...
                             ...                        
220    psheoas sheeor qoepsheody odar ocpheo-opar ysa...
221    okoramog okar!y darar oky salo oro-ain okor sa...
222    otaramy otoldy otor-chy oldy dar-ary otaly ols...
223    tydlo choly cthor orchey s shy odaiin sary-sho...
224    fochor oporody opy shor daiin qopchypcho qofol...
Name: text, Length: 225, dtype: object

In [175]:

def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True,min_len=3)
        final.append(new)
    return (final)

data_words = gen_words(list(data))

print (data_words[0][0:20])

['char', 'sheod', 'qocpheeckhy', 'shodol', 'cthdaoto', 'qeos', 'sheey', 'chcthso', 'dsheor', 'cthey', 'qokeey', 'oteey', 'ykeeodain', 'sorary', 'daiin', 'daiin', 'deeamshol', 'shor', 'chkeey', 'qoteey']


In [176]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 3), (9, 1), (10, 1), (11, 1), (12, 8), (13, 1), (14, 2), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1)]
alcfhy


In [177]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=50,
                                           passes=50,
                                           iterations=150,
                                           alpha="auto")

In [178]:
lda_model.print_topics()

[(0,
  '0.009*"sheol" + 0.006*"cheol" + 0.006*"cheo" + 0.006*"cheor" + 0.006*"cthey" + 0.005*"okeol" + 0.004*"shey" + 0.004*"okeey" + 0.004*"sheor" + 0.004*"ctheey"'),
 (1,
  '0.031*"shedy" + 0.028*"chedy" + 0.023*"qokeedy" + 0.022*"qokain" + 0.021*"qokedy" + 0.017*"qokeey" + 0.015*"shey" + 0.014*"chey" + 0.013*"qol" + 0.012*"qokal"'),
 (2,
  '0.052*"daiin" + 0.029*"chol" + 0.022*"chor" + 0.015*"shol" + 0.012*"cthy" + 0.011*"sho" + 0.010*"chy" + 0.010*"dar" + 0.009*"dain" + 0.008*"dal"'),
 (3,
  '0.026*"aiin" + 0.021*"daiin" + 0.011*"dar" + 0.010*"chedy" + 0.009*"dal" + 0.009*"chdy" + 0.008*"okaiin" + 0.008*"qokaiin" + 0.007*"chey" + 0.007*"chol"'),
 (4,
  '0.002*"chal" + 0.001*"choeey" + 0.001*"keeody" + 0.001*"lkchey" + 0.001*"ysheod" + 0.001*"cheokam" + 0.001*"ldain" + 0.001*"chaly" + 0.001*"okalal" + 0.001*"cpheody"')]

In [119]:
qq=data[1]

In [151]:
datadf.iloc[2,0]

'101r1'

In [179]:
doctopics=[]
datadf
k=0
for a in data:

    bow = id2word.doc2bow(a.split())
    t = lda_model.get_document_topics(bow,per_word_topics=False,minimum_probability=0.33)
    t.append(datadf.iloc[k,0])
    k=k+1
    doctopics.append(t)
    


In [180]:
doctopics

[[(2, 0.71747744), '100r'],
 [(3, 0.87269557), '100v'],
 [(0, 0.49030903), '101r1'],
 [(0, 0.3573261), (3, 0.40141678), '101v2'],
 [(3, 0.899806), '102r1'],
 [(3, 0.8728626), '102r2'],
 [(3, 0.9303472), '102v1'],
 [(0, 0.5209029), (3, 0.47718185), '102v2'],
 [(1, 0.85998356), '103r'],
 [(1, 0.5432537), (3, 0.4561548), '103v'],
 [(3, 0.99924433), '104r'],
 [(3, 0.6082596), '104v'],
 [(3, 0.65526944), '105r'],
 [(3, 0.74945587), '105v'],
 [(3, 0.6358427), '106r'],
 [(3, 0.99928546), '106v'],
 [(3, 0.8567707), '107r'],
 [(1, 0.50025123), (3, 0.49916735), '107v'],
 [(1, 0.64956105), (3, 0.34991026), '108r'],
 [(1, 0.83913726), '108v'],
 [(2, 0.99451464), '10r'],
 [(2, 0.99245167), '10v'],
 [(1, 0.7857528), '111r'],
 [(1, 0.8006426), '111v'],
 [(1, 0.5686472), (3, 0.43063343), '112r'],
 [(3, 0.81186), '112v'],
 [(3, 0.7168664), '113r'],
 [(3, 0.86657643), '113v'],
 [(3, 0.99928087), '114r'],
 [(3, 0.9991078), '114v'],
 [(1, 0.5147842), (3, 0.4121287), '115r'],
 [(3, 0.4371528), '115v'],
 [(

In [102]:
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds="mmds", R=20)
vis

c:\_projects\final_project_challenge\fpenv\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.341609 -0.108936       1        1  34.615080
5     -0.108929 -0.296040       2        1  25.029044
3     -0.202630  0.225371       3        1  23.994014
6      0.244587 -0.204724       4        1  13.884004
7      0.127124  0.061072       5        1   0.855636
8      0.083506  0.099024       6        1   0.599870
4      0.044623  0.089338       7        1   0.466099
9      0.056375  0.026206       8        1   0.408398
0      0.044559  0.053810       9        1   0.119143
2      0.052393  0.054880      10        1   0.028713, topic_info=         Term       Freq       Total Category  logprob  loglift
614     chedy  536.00000  536.000000  Default  20.0000  20.0000
784     shedy  485.00000  485.000000  Default  19.0000  19.0000
28      daiin  808.00000  808.000000  Default  18.0000  18.0000
23       chol  346.00000  346.000000  Default  17.0000  17.0000
741    qokedy  280.00000  280.000000  Default  16.0000  16.0000
...       ...        ...         ...      ...      ...      ...
2712  aithchr    0.00137    1.125024  Topic10  -8.9084   1.4445
2713     akal    0.00137    1.125024  Topic10  -8.9084   1.4445
2714   alkain    0.00137    1.125024  Topic10  -8.9084   1.4445
2715  aloiiin    0.00137    1.125024  Topic10  -8.9084   1.4445
2716    araii    0.00137    1.125024  Topic10  -8.9084   1.4445

[300 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
155       1  0.217818    aiin
155       2  0.239600    aiin
155       3  0.542566    aiin
931       3  0.995512     air
2711      3  0.888870   airol
...     ...       ...     ...
3218      4  0.952438  ytchey
93        5  0.965577  ytchol
476       5  0.503282   yteor
3564      6  0.623123     yto
423       5  0.947537    ytol

[272 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 6, 4, 7, 8, 9, 5, 10, 1, 3])